### Azure agent 

In [ ]:
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition
from custom_tools.legi_fr_tools import get_t_p, ping_pong_test_p
from azure.ai.agents.models import FunctionTool


from dotenv import load_dotenv
import os 

load_dotenv()

True

In [ ]:
# Foundry Project endpoint
project_client = AIProjectClient(
    endpoint=os.environ.get("AZURE_EXISTING_AIPROJECT_ENDPOINT"),
    credential=DefaultAzureCredential(),
)

In [ ]:
# retreive data on agents from our Azure project 

agents = list(project_client.agents.list())

if not agents:
    print("No agent found in the Azure project")
else:
    print(f"{len(agents)} Agent(s) found in the our project :\n")
    
    for agent in agents:
        print("-" * 40)
        print(f"Agent name : {agent.name}")
        print(f"Instruction : {agent.versions.latest.definition.instructions}")
        print(f"Model : {agent.versions.latest.definition.model}")
        print(f"Description : {agent.versions.latest.description}")
        
        tools = agent.versions.latest.definition.tools
        
        if tools: 
            for i in range(len(tools)): 
                if agent.versions.latest.definition.tools : 
                    print(f"Tool {i} :  {agent.versions.latest.definition.tools[i].get('type')} - {agent.versions.latest.definition.tools[i].get('server_label')}")




3 agent(s) trouvé(s) dans le projet :

----------------------------------------
Agent name : legal-agent-fr
Instruction : 
Model : gpt-4.1-mini
Description : 
----------------------------------------
Agent name : ds-agent
Instruction : You are an experienced  Data scientist , AI engineer with a high expertise giving precise and helpful information.
Model : gpt-5-mini
Description : 
Tool 0 :  mcp - HuggingFaceMCPServer
Tool 1 :  mcp - GitHub
Tool 2 :  mcp - MicrosoftLearn
Tool 3 :  code_interpreter - None
Tool 4 :  mcp - MicrosoftFoundryRemoteMCP
----------------------------------------
Agent name : test-agent
Instruction : 
Model : gpt-4.1-mini
Description : 


In [17]:
openai_client = project_client.get_openai_client()

In [ ]:
# tester un agent donné 

user_question="comment tu évalues ton savoir et tes capacités en matière de droit des assurances"

response = openai_client.responses.create(
    input=[{"role": "user", "content": user_question}],
    extra_body={"agent": {"name": agent.name, "type": "agent_reference"}},
)
print(f"Response output: {response.output_text}")

Response output: Bonne question — voici comment j’évalue mon savoir et mes capacités en droit des assurances (en droit français) :

Ce que je maîtrise
- Bases et principes : bonnes connaissances du Code des assurances et des principes clés (devoir d’information et de conseil, déclaration du risque, aggravation/variation du risque, nullité, prescription, subrogation, recours, garantie et exclusion).
- Contrats et produits : capacité à expliquer et analyser les clauses d’assurances courantes (RC, auto, habitation, assurance emprunteur, assurance-vie, dommage-ouvrage / décennale, protection juridique).
- Procédure et contentieux : compréhension des voies d’action (mise en demeure, saisine du médiateur, référé, action en responsabilité, délais et prescriptions) et des conséquences pratiques d’actes procéduraux.
- Jurisprudence et doctrine : synthèse d’arrêts majeurs et application des principes à des cas types (jusqu’à ma date de connaissance).
- Assistance pratique : rédaction/relève de c

In [ ]:
list_legifrance_tools={get_t_p, ping_pong_test_p}
functions_tool = FunctionTool(list_legifrance_tools)


In [ ]:
# Creates a new agent version, bumps the agent version if parameters have changed
agent = project_client.agents.create_version(  
    agent_name="legal-agent-fr",
    definition=PromptAgentDefinition(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME"),
            instructions="",
            temperature=0,
            #Reasoning=,
            tools= functions_tool.definitions
        ),
)